In [1]:
from functools import partial
from typing import List, Tuple

import numpy as np
from ipywidgets import widgets, HBox, VBox, Layout
from IPython.display import HTML, display, update_display

In [2]:
class Colors:
    START = 'green'
    GOAL = 'blue'
    OBSTACLE = 'black'
    PATH = 'red'


class Interface:
    def __init__(self, shape=(10,10)):
        self.n_rows, self.n_cols = shape
        self.buttons = [
            [
                widgets.Button(
                    description='',
                    layout=Layout(width='50px', height='50px')
                )
                for _ in range(self.n_cols)
            ]
            for _ in range(self.n_rows)
        ]

        self.map_widget = VBox([HBox(row) for row in self.buttons])
        self.link_positions()

        self.start_pos, self.goal_pos = None, None
        self.obstacles = set()

    def on_select_position(self, pos: tuple, button: widgets.Button):
        """Callback on user click on a position of the map

        :param pos: row, column of button on grid
        :param button: clicked button
        """
        if self.start_pos is None:
            self.start_pos = pos
            button.style.button_color = Colors.START

        elif self.goal_pos is None:
            self.goal_pos = pos
            button.style.button_color = Colors.GOAL

        else:
            self.obstacles.add(pos)
            button.style.button_color = Colors.OBSTACLE

    def link_positions(self):
        """Link clicks on buttons"""
        for i, row in enumerate(self.buttons):
            for j, button in enumerate(row):
                button.on_click(partial(self.on_select_position, (i, j)))

    def disable_buttons(self):
        """Disable buttons to avoid clicks after end game"""
        for row in self.buttons:
            for button in row:
                button.disabled = True

    def start(self):
        """Display map interface"""
        display(self.map_widget)



In [3]:
interface = Interface()
interface.start()

# BFS

## Parte 1 -
- Implementar regra para obter movimentações possíveis (cima, baixo, direita e esquerda)
- Implementar função que remove movimentos proibidos (fora do mapa ou sobre obstáculos)

## Parte 2   -Implementar busca em largura (breadth first search - BFS)
1. Receba o mapa
2. Crie uma lista de nós a visitar 3. (iniciada com a posição de inicio)
3. Crie uma lista de nós a serem visitados
4. Enquanto houverem posições a serem visitadas:
5. Obtenha a primeira
6. Verifique se esse nó já foi visitado, se sim pule
7. Verifique se esse nó é o objetivo, se sim retorne
8. Adicione esse nó a lista de visitados
9. Adicione ao fim da lista a ser visitado os filhos dele (movimentos possíveis)
Adicione a cada nó o caminho até ele
Edite o retorno, para retornar o caminho


In [14]:
# MOVES = quais são os movimentos possíveis


def BFS(start_pos: Tuple[int, int], goal_pos: Tuple[int, int],
        obstacles: List[Tuple[int, int]], max_y: int, max_x: int):
    """Breadth-first search algorithm"""

    def is_valid(position: Tuple[int, int]):
        """Checks if a position is valid
        i.e. it is inside map and it is not a obstacle"""
        # Check if position is within the grid boundaries and not an obstacle
        return 0 <= position[0] < max_y and 0 <= position[1] < max_x and position not in obstacles


    def get_next_positions(y, x):
        """Get next possible moves"""
        # Define possible moves (up, down, left, right)
        moves = [(0, 1), (0, -1), (1, 0), (-1, 0)]
        # Generate next positions
        next_positions = [(y + dy, x + dx) for dy, dx in moves]
        # Filter out invalid positions
        valid_positions = [pos for pos in next_positions if is_valid(pos)]
        return valid_positions

    queue = [(start_pos, [start_pos])]
    visited = set()

    while queue:
        current_pos, path = queue.pop(0)  # Get the first element from the queue

        if current_pos == goal_pos:
            return path  # Goal found, return the path

        if current_pos in visited:
            continue  # Skip if already visited

        visited.add(current_pos)

        for next_pos in get_next_positions(current_pos[0], current_pos[1]):
            queue.append((next_pos, path + [next_pos]))  # Add neighbors to the queue
            # Removed the return None statement from here to allow the search to continue
    # If no path is found after the loop completes, then return None
    return None

In [16]:
class PathFinder(Interface):
    def __init__(self, shape=(10, 10)):
        super().__init__(shape)
        self.find_button = widgets.Button(
            description='Find shortest path',
            style=widgets.ButtonStyle(button_color=Colors.START)
        )

        self.find_button.on_click(self.search)

    def set_path(self, path: List[Tuple[int, int]]):
        """Color a path in map"""
        for y, x in path[1:]:
            self.buttons[y][x].style.button_color = Colors.PATH

    def search(self, *args):
        """Disable editions on map, then finds the shorterst path and draw it"""
        self.disable_buttons()
        path = BFS(self.start_pos, self.goal_pos, self.obstacles, self.n_rows,
                   self.n_cols)
        self.set_path(path)
        return path

    def start(self):
        """Display map interface"""
        display(VBox([self.map_widget, self.find_button]))



path_finder = PathFinder()
path_finder.start()